In [144]:
import pandas as pd 
import numpy as np 
import math

class Node:
    def __init__(self, ids = [],  depth = 0, children = [], entropy = 0):
#        print(ids[:10])
        self.ids = ids
        self.depth = depth
        self.children = children
        self.entropy = entropy
        self.attributes = None
        self.label = None 
        self.value_order = None

    def setAttribute(self, attributes, value_order):
        self.attributes = attributes
        self.value_order = value_order

    def setLabel(self, label):
        self.lable = label

class DecisionTree:
    def __init__(self, depthNode = 20, thredhold = 0.05):
        self.depthNode = depthNode
        self.thredhold = thredhold

    def fit(self, data, target):
        self.data = data
        self.attributes = list(data)
        self.target = target
        self.labels = target.unique()
        
        self.root = Node(ids= range(self.data.shape[0]),entropy= self.info_entropy(self.target), depth= 0)

        queue = [self.root]
        while queue:
            node = queue.pop()
            if node.depth < self.depthNode and node.entropy > self.thredhold:
                node.children = self.split(node)
                queue += node.children
            else:
                self.setLabel(node)
    
    def predict(self, X_train):
        y_predict = [None]*X_train.size
        for i in range(X_train.size):
            x = X_train.iloc[i,:]
            node = self.root
            while node.children:
                print(list(node.value_order), x, x[node.attributes])#node.value_order.index[x[node.attributes]])
                node = node.children[ list(node.value_order)[x[node.attributes]]]
            y_predict[i] = node.label
        return y_predict

    def info_entropy(self, rows):
        entropy = 0.0
#        print(rows.unique())
        for i in rows.unique():
            p = rows[rows == i].size / rows.size
#             print(p)
            entropy += - p * math.log(p)
        return entropy

    def split(self, node):
        ids = node.ids
        data = self.data.iloc[ids,:]
        best_gain = 0
        best_attribute = None
        best_split = []
        value_order = None
        for i,attr in enumerate(self.attributes):
            if attr != node.attributes:
                data_i = data.iloc[:, i]
                value_unique = data_i.unique().tolist()
                split_idx= []
                for value in value_unique:
                    sub_ids = data_i.index[data_i == value].tolist()
                    split_idx.append(sub_ids)
                entropy_sub = 0.0
                for sub_ids in split_idx:
                    entropy_sub += len(sub_ids)* self.info_entropy(self.target[sub_ids]) / len(ids)
                gain = node.entropy - entropy_sub
                if gain > best_gain:
                    best_gain = gain
                    best_attribute = attr
                    best_split = split_idx
                    value_order = value_unique
        node.setAttribute(best_attribute, value_order)
        childrenNode = [Node(ids = sub_ids, depth= node.depth + 1, entropy=self.info_entropy(self.target[sub_ids])) for sub_ids in best_split]
        return childrenNode


    def setLabel(self, node):
        ids = node.ids
        node.setLabel(self.target[ids].value_counts()[0])

In [153]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

dataset = pd.read_csv("weather.csv")
X = dataset.drop('play', axis = 1)
y = dataset['play']
dataset.head

<bound method NDFrame.head of     id   outlook temperature humidity    wind play
0    1     sunny         hot     high    weak   no
1    2     sunny         hot     high  strong   no
2    3  overcast         hot     high    weak  yes
3    4     rainy        mild     high    weak  yes
4    5     rainy        cool   normal    weak  yes
5    6     rainy        cool   normal  strong   no
6    7  overcast        cool   normal  strong  yes
7    8     sunny        mild     high    weak   no
8    9     sunny        cool   normal    weak  yes
9   10     rainy        mild   normal    weak  yes
10  11     sunny        mild   normal  strong  yes
11  12  overcast        mild     high  strong  yes
12  13  overcast         hot   normal    weak  yes
13  14     rainy        mild     high  strong   no>

In [154]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [147]:
classifier = DecisionTree()
classifier.fit(X_train,y_train)

In [148]:
y_pred = classifier.predict(X_test)

[8, 7, 12, 3, 9, 1, 13, 14, 10, 5, 2] id                  6
outlook         rainy
temperature      cool
humidity       normal
wind           strong
Name: 5, dtype: object 6


IndexError: list index out of range

In [135]:
X_train.shape

(11, 5)

In [159]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)

ValueError: could not convert string to float: 'strong'

,id,outlook,temperature,humidity,wind
6,7,overcast,cool,normal,strong
11,12,overcast,mild,high,strong
1,2,sunny,hot,high,strong
10,11,sunny,mild,normal,strong
4,5,rainy,cool,normal,weak
8,9,sunny,cool,normal,weak
3,4,rainy,mild,high,weak
7,8,sunny,mild,high,weak
9,10,rainy,mild,normal,weak
5,6,rainy,cool,normal,strong
